In [ ]:
import cv2
import numpy as np
import os
import joblib
import pandas as pd
import time

# --- Configuration ---
image_path =  "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data/00443870.png"   
model_filename = "svm_line_classifier.joblib"  # Filename of your saved SVM model
scaler_filename = "scaler_line_features.joblib"  # Filename of your saved StandardScaler
# Removed: training_csv_path = "line_detection_features.csv" # No longer needed!

# --- USER-DEFINED OPTIMIZED PARAMETERS ---
# IMPORTANT: REPLACE THESE VALUES with the "perfect variables" you found during tuning!
OPTIMIZED_PARAMS = {
    "lower_L": 137,
    "upper_L": 255,
    "lower_A": 134,
    "upper_A": 161,
    "lower_B": 138,
    "upper_B": 165,
    "color_morph_kernel_size": 3,
    "edge_morph_kernel_size": 7,
    "canny_thresh1": 18,
    "canny_thresh2": 66,
    "hough_threshold": 57,       # Min votes for a line
    "hough_min_length": 18,      # Min line length
    "hough_max_gap": 17,         # Max gap to connect segments
    "crop_percent": 55,
    "line_center_tolerance_percent": 10 # Percentage of image width for "center" tolerance
}

# --- 1. Load the Model and Scaler ---
try:
    loaded_model = joblib.load(model_filename)
    loaded_scaler = joblib.load(scaler_filename)
    print("SVM model and scaler loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model or scaler file not found. Ensure '{model_filename}' and '{scaler_filename}' are in the correct directory.")
    exit()
except Exception as e:
    print(f"Error loading model or scaler: {e}")
    exit()

# --- CRITICAL FIX: Get the feature column order directly from the loaded scaler ---
# The scaler stores the feature names it was fitted on, ensuring correct order.
feature_columns_order = loaded_scaler.feature_names_in_.tolist()
print("Successfully retrieved feature column order from the loaded scaler.")


# --- 2. Load the Image ---
image = cv2.imread(image_path)
if image is None:
    print(f"Error: Could not read image at '{image_path}'. Please check the path and file integrity.")
    exit()

print(f"\nProcessing image: {os.path.basename(image_path)}")

# --- 3. Image Processing and Feature Extraction (Identical to training data generation) ---

# --- 3.1. Crop from top ---
crop_y = int(image.shape[0] * OPTIMIZED_PARAMS["crop_percent"] / 100)
if crop_y >= image.shape[0]:
    crop_y = image.shape[0] - 1
cropped_image = image[crop_y:, :].copy()

# Initialize a dictionary to hold features for this single image
current_features = {}


start_time = time.perf_counter()

# Handle cases where cropped image might be empty
if cropped_image.shape[0] == 0 or cropped_image.shape[1] == 0:
    print(f"Warning: Cropped image for {os.path.basename(image_path)} is empty. Defaulting to 'No Line Detected' features.")
    # Fill with default/zero features as if no line was detected
    current_features = {
        "cx": -1,
        "num_detected_lines": 0,
        "avg_line_length": 0,
        "total_line_length": 0,
        "std_line_length": 0,
        "avg_line_angle_deg": 0,
        "std_line_angle_deg": 0,
        "line_cx_mean": -1,
        "line_cx_std": 0,
        "line_cy_mean": -1,
        "longest_line_length": 0,
        "longest_line_angle_deg": 0,
        "mask_pixel_count": 0,
        "mask_area_ratio": 0,
        "mask_centroid_x_norm": 0.5,
        "mask_centroid_y_norm": 0.5,
        "mask_hu_moment_1": 0, "mask_hu_moment_2": 0, "mask_hu_moment_3": 0,
        "mask_hu_moment_4": 0, "mask_hu_moment_5": 0, "mask_hu_moment_6": 0, "mask_hu_moment_7": 0,
        "color_mask_pixel_count": 0,
        "color_mask_area_ratio": 0,
        "is_line_detected_binary": 0
    }
else:
   
    lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_cropped)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l_channel)
    lab_eq = cv2.merge([l_eq, a_channel, b_channel])
    blurred_lab_eq = cv2.medianBlur(lab_eq, 5)

    lower_orange_lab = np.array([OPTIMIZED_PARAMS["lower_L"], OPTIMIZED_PARAMS["lower_A"], OPTIMIZED_PARAMS["lower_B"]])
    upper_orange_lab = np.array([OPTIMIZED_PARAMS["upper_L"], OPTIMIZED_PARAMS["upper_A"], OPTIMIZED_PARAMS["upper_B"]])
    color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

  
    color_morph_kernel = np.ones((OPTIMIZED_PARAMS["color_morph_kernel_size"], OPTIMIZED_PARAMS["color_morph_kernel_size"]), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)

    gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    blurred_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)
    edge_mask = cv2.Canny(blurred_gray, OPTIMIZED_PARAMS["canny_thresh1"], OPTIMIZED_PARAMS["canny_thresh2"])

   
    edge_morph_kernel = np.ones((OPTIMIZED_PARAMS["edge_morph_kernel_size"], OPTIMIZED_PARAMS["edge_morph_kernel_size"]), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)


    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180,
                                OPTIMIZED_PARAMS["hough_threshold"],
                                minLineLength=OPTIMIZED_PARAMS["hough_min_length"],
                                maxLineGap=OPTIMIZED_PARAMS["hough_max_gap"])

    
    # Initialize line features
    num_detected_lines = 0
    avg_line_length = 0
    total_line_length = 0
    std_line_length = 0
    avg_line_angle_deg = 0
    std_line_angle_deg = 0
    line_cx_mean = -1
    line_cx_std = 0
    line_cy_mean = -1
    longest_line_length = 0
    longest_line_angle_deg = 0
    is_line_detected_binary = 0
    cx = -1 
    if lines is not None:
        is_line_detected_binary = 1
        num_detected_lines = len(lines)

        all_line_midpoints_x = []
        all_line_midpoints_y = []
        line_lengths = []
        line_angles_rad = []

        max_length_found = 0
        angle_of_longest_line = 0

        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            line_lengths.append(length)

            if length > max_length_found:
                max_length_found = length
                angle_of_longest_line = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

            angle_rad = np.arctan2(y2 - y1, x2 - x1)
            line_angles_rad.append(angle_rad)

            mid_x = (x1 + x2) // 2
            mid_y = (y1 + y2) // 2
            all_line_midpoints_x.append(mid_x)
            all_line_midpoints_y.append(mid_y)

        if line_lengths:
            avg_line_length = np.mean(line_lengths)
            total_line_length = np.sum(line_lengths)
            std_line_length = np.std(line_lengths) if len(line_lengths) > 1 else 0
            longest_line_length = max_length_found
            longest_line_angle_deg = angle_of_longest_line

        if all_line_midpoints_x:
            cx = int(np.mean(all_line_midpoints_x)) # This cx is used for the label mapping
            line_cx_mean = cx
            line_cx_std = np.std(all_line_midpoints_x) if len(all_line_midpoints_x) > 1 else 0
            line_cy_mean = int(np.mean(all_line_midpoints_y))

        if line_angles_rad:
            normalized_angles_deg = [angle % 180 for angle in np.degrees(line_angles_rad)]
            avg_line_angle_deg = np.mean(normalized_angles_deg)
            std_line_angle_deg = np.std(normalized_angles_deg) if len(normalized_angles_deg) > 1 else 0

    # Features from Final Mask
    mask_pixel_count = np.sum(final_mask > 0)
    mask_area_ratio = mask_pixel_count / (final_mask.shape[0] * final_mask.shape[1]) if (final_mask.shape[0] * final_mask.shape[1]) > 0 else 0

    M = cv2.moments(final_mask)
    mask_centroid_x_norm = 0.5
    mask_centroid_y_norm = 0.5
    hu_moments = np.zeros(7) # Initialize to zeros
    if M["m00"] != 0:
        mask_centroid_x = M["m10"] / M["m00"]
        mask_centroid_y = M["m01"] / M["m00"]
        mask_centroid_x_norm = mask_centroid_x / final_mask.shape[1]
        mask_centroid_y_norm = mask_centroid_y / final_mask.shape[0]
        hu_moments = cv2.HuMoments(M).flatten()
    
    # Features from Morphed Color Mask
    color_mask_pixel_count = np.sum(color_mask_morphed > 0)
    color_mask_area_ratio = color_mask_pixel_count / (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) if (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) > 0 else 0

    # Store all features for this image in a dictionary
    current_features = {
        "cx": cx,
        "num_detected_lines": num_detected_lines,
        "avg_line_length": avg_line_length,
        "total_line_length": total_line_length,
        "std_line_length": std_line_length,
        "avg_line_angle_deg": avg_line_angle_deg,
        "std_line_angle_deg": std_line_angle_deg,
        "line_cx_mean": line_cx_mean,
        "line_cx_std": line_cx_std,
        "line_cy_mean": line_cy_mean,
        "longest_line_length": longest_line_length,
        "longest_line_angle_deg": longest_line_angle_deg,
        "mask_pixel_count": mask_pixel_count,
        "mask_area_ratio": mask_area_ratio,
        "mask_centroid_x_norm": mask_centroid_x_norm,
        "mask_centroid_y_norm": mask_centroid_y_norm,
        "mask_hu_moment_1": hu_moments[0], "mask_hu_moment_2": hu_moments[1], "mask_hu_moment_3": hu_moments[2],
        "mask_hu_moment_4": hu_moments[3], "mask_hu_moment_5": hu_moments[4], "mask_hu_moment_6": hu_moments[5], "mask_hu_moment_7": hu_moments[6],
        "color_mask_pixel_count": color_mask_pixel_count,
        "color_mask_area_ratio": color_mask_area_ratio,
        "is_line_detected_binary": is_line_detected_binary
    }

# Convert the dictionary to a DataFrame, ensuring the correct column order
input_features = pd.DataFrame([current_features])
# CRITICAL FIX: Reindex to match training order, using feature_columns_order from the scaler
input_features = input_features[feature_columns_order]

# --- 4. Scale the Features ---
# Use the loaded scaler to transform the new input features
input_features_scaled = loaded_scaler.transform(input_features)

# --- 5. Make the Prediction ---
# The predict method returns an array, so take the first element [0]
prediction = loaded_model.predict(input_features_scaled)[0]

# --- 6. Output the Prediction ---
# Map the numerical prediction back to a human-readable label
label_map = {-1: "Line on Left", 0: "Line in Center", 1: "Line on Right", 2: "No Line Detected"}
predicted_label_text = label_map.get(prediction, "Unknown Label")
end_time = time.perf_counter()

# --- Calculate and print runtime ---
runtime_ms = (end_time - start_time) * 1000
print(f"\nPrediction runtime (including feature extraction & scaling): {runtime_ms:.2f} ms")


print(f"\nPrediction for '{os.path.basename(image_path)}':")
print(f"  Predicted Class: {predicted_label_text}")
print(f"  Numerical Label: {prediction}")



SVM model and scaler loaded successfully.
Successfully retrieved feature column order from the loaded scaler.

Processing image: 00443870.png

Prediction runtime (including feature extraction & scaling): 40.15 ms

Prediction for '00443870.png':
  Predicted Class: Line on Right
  Numerical Label: 1


In [ ]:
# Import libraries and load model
import cv2
import numpy as np
import os
import joblib
import pandas as pd
import time
from IPython.display import display
from ipywidgets import interact, widgets

model_filename = "svm_line_classifier.joblib"
scaler_filename = "scaler_line_features.joblib"

OPTIMIZED_PARAMS = {
    "lower_L": 137,
    "upper_L": 255,
    "lower_A": 134,
    "upper_A": 161,
    "lower_B": 138,
    "upper_B": 165,
    "color_morph_kernel_size": 3,
    "edge_morph_kernel_size": 7,
    "canny_thresh1": 18,
    "canny_thresh2": 66,
    "hough_threshold": 57,
    "hough_min_length": 18,
    "hough_max_gap": 17,
    "crop_percent": 55,
    "line_center_tolerance_percent": 10
}

try:
    loaded_model = joblib.load(model_filename)
    loaded_scaler = joblib.load(scaler_filename)
    print("SVM model and scaler loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model or scaler file not found. Ensure '{model_filename}' and '{scaler_filename}' are in the correct directory.")
except Exception as e:
    print(f"Error loading model or scaler: {e}")

feature_columns_order = loaded_scaler.feature_names_in_.tolist()
print("Successfully retrieved feature column order from the loaded scaler.")


SVM model and scaler loaded successfully.
Successfully retrieved feature column order from the loaded scaler.


In [ ]:
# Image selection
def get_images_from_directory(directory_path):
    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']
    images = []
    if os.path.exists(directory_path):
        for file in os.listdir(directory_path):
            if any(file.lower().endswith(ext) for ext in image_extensions):
                images.append(os.path.join(directory_path, file))
    return sorted(images)

dataset_path = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data/"
available_images = get_images_from_directory(dataset_path)

if available_images:
    image_dropdown = widgets.Dropdown(
        options=[(os.path.basename(img), img) for img in available_images],
        description='Select Image:',
        style={'description_width': 'initial'}
    )
    display(image_dropdown)
    print(f"Found {len(available_images)} images in the directory.")
else:
    print("No images found in the specified directory.")
    image_dropdown = widgets.Text(
        placeholder='Enter full path to image file',
        description='Image Path:',
        style={'description_width': 'initial'}
    )
    display(image_dropdown)

Dropdown(description='Select Image:', options=(('00420478.png', 'C:/Users/BCI-Lab/Downloads/teamA_dataset/_out…

Found 1291 images in the directory.


In [ ]:
#Process selected image
def process_image():
    if hasattr(image_dropdown, 'value'):
        if isinstance(image_dropdown.value, str) and image_dropdown.value:
            image_path = image_dropdown.value
        else:
            print("Please select an image first.")
            return
    else:
        print("Please run Cell 2 first to select an image.")
        return
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at '{image_path}'. Please check the path and file integrity.")
        return

    print(f"\nProcessing image: {os.path.basename(image_path)}")

    crop_y = int(image.shape[0] * OPTIMIZED_PARAMS["crop_percent"] / 100)
    if crop_y >= image.shape[0]:
        crop_y = image.shape[0] - 1
    cropped_image = image[crop_y:, :].copy()

    current_features = {}
    start_time = time.perf_counter()

    if cropped_image.shape[0] == 0 or cropped_image.shape[1] == 0:
        print(f"Warning: Cropped image for {os.path.basename(image_path)} is empty. Defaulting to 'No Line Detected' features.")
        current_features = {
            "cx": -1,
            "num_detected_lines": 0,
            "avg_line_length": 0,
            "total_line_length": 0,
            "std_line_length": 0,
            "avg_line_angle_deg": 0,
            "std_line_angle_deg": 0,
            "line_cx_mean": -1,
            "line_cx_std": 0,
            "line_cy_mean": -1,
            "longest_line_length": 0,
            "longest_line_angle_deg": 0,
            "mask_pixel_count": 0,
            "mask_area_ratio": 0,
            "mask_centroid_x_norm": 0.5,
            "mask_centroid_y_norm": 0.5,
            "mask_hu_moment_1": 0, "mask_hu_moment_2": 0, "mask_hu_moment_3": 0,
            "mask_hu_moment_4": 0, "mask_hu_moment_5": 0, "mask_hu_moment_6": 0, "mask_hu_moment_7": 0,
            "color_mask_pixel_count": 0,
            "color_mask_area_ratio": 0,
            "is_line_detected_binary": 0
        }
    else:
        lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab_cropped)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        l_eq = clahe.apply(l_channel)
        lab_eq = cv2.merge([l_eq, a_channel, b_channel])
        blurred_lab_eq = cv2.medianBlur(lab_eq, 5)

        lower_orange_lab = np.array([OPTIMIZED_PARAMS["lower_L"], OPTIMIZED_PARAMS["lower_A"], OPTIMIZED_PARAMS["lower_B"]])
        upper_orange_lab = np.array([OPTIMIZED_PARAMS["upper_L"], OPTIMIZED_PARAMS["upper_A"], OPTIMIZED_PARAMS["upper_B"]])
        color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

        color_morph_kernel = np.ones((OPTIMIZED_PARAMS["color_morph_kernel_size"], OPTIMIZED_PARAMS["color_morph_kernel_size"]), np.uint8)
        color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
        color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1)
        color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)

        gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        blurred_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)
        edge_mask = cv2.Canny(blurred_gray, OPTIMIZED_PARAMS["canny_thresh1"], OPTIMIZED_PARAMS["canny_thresh2"])

        edge_morph_kernel = np.ones((OPTIMIZED_PARAMS["edge_morph_kernel_size"], OPTIMIZED_PARAMS["edge_morph_kernel_size"]), np.uint8)
        edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

        final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

        lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180,
                                    OPTIMIZED_PARAMS["hough_threshold"],
                                    minLineLength=OPTIMIZED_PARAMS["hough_min_length"],
                                    maxLineGap=OPTIMIZED_PARAMS["hough_max_gap"])

        num_detected_lines = 0
        avg_line_length = 0
        total_line_length = 0
        std_line_length = 0
        avg_line_angle_deg = 0
        std_line_angle_deg = 0
        line_cx_mean = -1
        line_cx_std = 0
        line_cy_mean = -1
        longest_line_length = 0
        longest_line_angle_deg = 0
        is_line_detected_binary = 0
        cx = -1

        if lines is not None:
            is_line_detected_binary = 1
            num_detected_lines = len(lines)

            all_line_midpoints_x = []
            all_line_midpoints_y = []
            line_lengths = []
            line_angles_rad = []

            max_length_found = 0
            angle_of_longest_line = 0

            for line in lines:
                x1, y1, x2, y2 = line[0]
                length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                line_lengths.append(length)

                if length > max_length_found:
                    max_length_found = length
                    angle_of_longest_line = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

                angle_rad = np.arctan2(y2 - y1, x2 - x1)
                line_angles_rad.append(angle_rad)

                mid_x = (x1 + x2) // 2
                mid_y = (y1 + y2) // 2
                all_line_midpoints_x.append(mid_x)
                all_line_midpoints_y.append(mid_y)

            if line_lengths:
                avg_line_length = np.mean(line_lengths)
                total_line_length = np.sum(line_lengths)
                std_line_length = np.std(line_lengths) if len(line_lengths) > 1 else 0
                longest_line_length = max_length_found
                longest_line_angle_deg = angle_of_longest_line

            if all_line_midpoints_x:
                cx = int(np.mean(all_line_midpoints_x))
                line_cx_mean = cx
                line_cx_std = np.std(all_line_midpoints_x) if len(all_line_midpoints_x) > 1 else 0
                line_cy_mean = int(np.mean(all_line_midpoints_y))

            if line_angles_rad:
                normalized_angles_deg = [angle % 180 for angle in np.degrees(line_angles_rad)]
                avg_line_angle_deg = np.mean(normalized_angles_deg)
                std_line_angle_deg = np.std(normalized_angles_deg) if len(normalized_angles_deg) > 1 else 0

        mask_pixel_count = np.sum(final_mask > 0)
        mask_area_ratio = mask_pixel_count / (final_mask.shape[0] * final_mask.shape[1]) if (final_mask.shape[0] * final_mask.shape[1]) > 0 else 0

        M = cv2.moments(final_mask)
        mask_centroid_x_norm = 0.5
        mask_centroid_y_norm = 0.5
        hu_moments = np.zeros(7)
        if M["m00"] != 0:
            mask_centroid_x = M["m10"] / M["m00"]
            mask_centroid_y = M["m01"] / M["m00"]
            mask_centroid_x_norm = mask_centroid_x / final_mask.shape[1]
            mask_centroid_y_norm = mask_centroid_y / final_mask.shape[0]
            hu_moments = cv2.HuMoments(M).flatten()
        
        color_mask_pixel_count = np.sum(color_mask_morphed > 0)
        color_mask_area_ratio = color_mask_pixel_count / (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) if (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) > 0 else 0

        current_features = {
            "cx": cx,
            "num_detected_lines": num_detected_lines,
            "avg_line_length": avg_line_length,
            "total_line_length": total_line_length,
            "std_line_length": std_line_length,
            "avg_line_angle_deg": avg_line_angle_deg,
            "std_line_angle_deg": std_line_angle_deg,
            "line_cx_mean": line_cx_mean,
            "line_cx_std": line_cx_std,
            "line_cy_mean": line_cy_mean,
            "longest_line_length": longest_line_length,
            "longest_line_angle_deg": longest_line_angle_deg,
            "mask_pixel_count": mask_pixel_count,
            "mask_area_ratio": mask_area_ratio,
            "mask_centroid_x_norm": mask_centroid_x_norm,
            "mask_centroid_y_norm": mask_centroid_y_norm,
            "mask_hu_moment_1": hu_moments[0], "mask_hu_moment_2": hu_moments[1], "mask_hu_moment_3": hu_moments[2],
            "mask_hu_moment_4": hu_moments[3], "mask_hu_moment_5": hu_moments[4], "mask_hu_moment_6": hu_moments[5], "mask_hu_moment_7": hu_moments[6],
            "color_mask_pixel_count": color_mask_pixel_count,
            "color_mask_area_ratio": color_mask_area_ratio,
            "is_line_detected_binary": is_line_detected_binary
        }

    input_features = pd.DataFrame([current_features])
    input_features = input_features[feature_columns_order]

    input_features_scaled = loaded_scaler.transform(input_features)

    prediction = loaded_model.predict(input_features_scaled)[0]

    label_map = {-1: "Line on Left", 0: "Line in Center", 1: "Line on Right", 2: "No Line Detected"}
    predicted_label_text = label_map.get(prediction, "Unknown Label")
    end_time = time.perf_counter()

    runtime_ms = (end_time - start_time) * 1000
    print(f"\nPrediction runtime (including feature extraction & scaling): {runtime_ms:.2f} ms")

    print(f"\nPrediction for '{os.path.basename(image_path)}':")
    print(f"  Predicted Class: {predicted_label_text}")
    print(f"  Numerical Label: {prediction}")


In [ ]:

process_image()


Processing image: 00425599.png

Prediction runtime (including feature extraction & scaling): 39.60 ms

Prediction for '00425599.png':
  Predicted Class: Line in Center
  Numerical Label: 0
